In [1]:
import numpy as np
import tensorflow as tf

In [2]:
h = 3
w = 3
n_in = 1
n_out = 1
# k_size = h
ksize = 3
n_channels = 1

In [3]:
def get_linear_ar_mask(n_in, n_out, zerodiagonal=False):
    assert n_in % n_out == 0 or n_out % n_in == 0, "%d - %d" % (n_in, n_out)

    mask = np.ones([n_in, n_out], dtype=np.float32)
    if n_out >= n_in:
        k = n_out // n_in
        for i in range(n_in):
            mask[i + 1:, i * k:(i + 1) * k] = 0
            if zerodiagonal:
                mask[i:i + 1, i * k:(i + 1) * k] = 0
    else:
        k = n_in // n_out
        for i in range(n_out):
            mask[(i + 1) * k:, i:i + 1] = 0
            if zerodiagonal:
                mask[i * k:(i + 1) * k:, i:i + 1] = 0
    return mask

In [4]:
def get_conv_ar_mask(h, w, n_in, n_out, zerodiagonal=False):
    """
    Function to get autoregressive convolution
    """
    l = (h - 1) // 2
    m = (w - 1) // 2
    mask = np.ones([h, w, n_in, n_out], dtype=np.float32)
    mask[:l, :, :, :] = 0
    mask[l, :m, :, :] = 0
    mask[l, m, :, :] = get_linear_ar_mask(n_in, n_out, zerodiagonal)
    return mask

In [5]:
def get_conv_square_ar_mask(h, w, n_in, n_out, zerodiagonal=False):
    """
    Function to get autoregressive convolution with square shape.
    """
    l = (h - 1) // 2
    m = (w - 1) // 2
    mask = np.ones([h, w, n_in, n_out], dtype=np.float32)
    mask[:l, :, :, :] = 0
    mask[:, :m, :, :] = 0
    mask[l, m, :, :] = get_linear_ar_mask(n_in, n_out, zerodiagonal)
    return mask

In [6]:
def get_conv_weight_np(filter_shape, stable_init=True, unit_testing=False):
    weight_np = np.random.randn(*filter_shape) * 0.02
    kcent = (filter_shape[0] - 1) // 2
    if stable_init or unit_testing:
        weight_np[kcent, kcent, :, :] += 1. * np.eye(filter_shape[3])
    weight_np = weight_np.astype('float32')
    return weight_np


In [7]:
mask_np = get_conv_square_ar_mask(ksize, ksize, n_channels, n_channels)
# print(mask_np[:,:,0,0])
mask_upsidedown_np = mask_np[::-1, ::-1, ::-1, ::-1].copy()
# print(mask_upsidedown_np[:,:,0,0])

mask_upsidedown = tf.constant(mask_upsidedown_np)
print(mask_upsidedown[:,:,0,0])

# filter_shape = [ksize, ksize, n_channels, n_channels]

# w_np = get_conv_weight_np(filter_shape)
# print(w_np.shape)

tf.Tensor(
[[1. 1. 0.]
 [1. 1. 0.]
 [0. 0. 0.]], shape=(3, 3), dtype=float32)


In [9]:
# w = tf.compat.v1.get_variable('W', dtype=tf.float32, initializer=w_np)
# b = tf.compat.v1.get_variable('b', [n_channels], initializer=tf.zeros_initializer())
# b = tf.reshape(b, [1, 1, 1, -1])
# # print(w)
# print(b)
# w = w * mask_upsidedown
# # print(w.shape,w)